# Environmental Justice and Health Inequalities: A Geospatial Analysis

This notebook walks through the analysis of air pollution, deprivation, and respiratory health outcomes in the UK. It explores the relationship between these factors and identifies areas of environmental injustice.

## 1. Setup and Data Loading

First, let's import the necessary libraries and load our datasets.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import geopandas as gpd
from scipy.stats import pearsonr, spearmanr
import os

# Set the plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Create output directories if they don't exist
os.makedirs('outputs', exist_ok=True)
os.makedirs('outputs/figures', exist_ok=True)
os.makedirs('outputs/data', exist_ok=True)

In [ ]:
# Load datasets
unified_df = pd.read_csv('unified_dataset_with_air_quality.csv')
health_df = pd.read_csv('health_indicators_by_lad.csv')
wards_df = pd.read_csv('Wards_December_2024_Boundaries_UK_BFC_2423639173483005972.csv')

print(f"Loaded {len(unified_df)} LSOA records, {len(health_df)} LAD health records, and {len(wards_df)} ward records.")

## 2. Exploratory Data Analysis

Let's explore the datasets to understand their structure and content.

In [ ]:
# Unified dataset summary
print("Unified Dataset Summary:")
print(f"Shape: {unified_df.shape}")
print("\nFirst few rows:")
unified_df.head()

In [ ]:
# Check for missing values
print("Missing values in unified dataset:")
missing_unified = unified_df.isnull().sum()
print(missing_unified[missing_unified > 0])

In [ ]:
# Health dataset summary
print("Health Dataset Summary:")
print(f"Shape: {health_df.shape}")
print("\nFirst few rows:")
health_df.head()

In [ ]:
# Basic statistics for key variables
print("Air Quality Statistics:")
air_quality_cols = ['NO2', 'O3', 'PM10', 'PM2.5']
unified_df[air_quality_cols].describe()

In [ ]:
print("Deprivation Statistics:")
deprivation_cols = ['imd_score_normalized', 'income_score_rate', 'employment_score_rate', 
                    'health_deprivation_and_disability_score']
unified_df[deprivation_cols].describe()

In [ ]:
print("Health Indicators Statistics:")
health_cols = ['chronic_conditions_normalized', 'asthma_diabetes_epilepsy_normalized', 
              'lrti_children_normalized', 'respiratory_health_index']
health_df[health_cols].describe()

## 3. Environmental Justice Analysis

Now, let's analyze the relationship between air pollution and deprivation to understand environmental justice issues.

In [ ]:
# Calculate correlation between air pollution and deprivation
pollution_cols = ['NO2_normalized', 'PM2.5_normalized', 'PM10_normalized', 'O3']
deprivation_cols = ['imd_score_normalized']

print("Correlation between air pollution and deprivation:")
for p_col in pollution_cols:
    for d_col in deprivation_cols:
        corr, p_value = pearsonr(unified_df[p_col], unified_df[d_col])
        print(f"{p_col} vs {d_col}: r = {corr:.3f}, p = {p_value:.6f}")

In [ ]:
# Create scatter plots
plt.figure(figsize=(16, 12))

for i, p_col in enumerate(pollution_cols):
    plt.subplot(2, 2, i+1)
    sns.scatterplot(x=unified_df[p_col], y=unified_df['imd_score_normalized'], 
                    alpha=0.5, edgecolor=None)
    
    # Add trend line
    sns.regplot(x=unified_df[p_col], y=unified_df['imd_score_normalized'], 
               scatter=False, line_kws={"color": "red"})
    
    plt.title(f'{p_col} vs IMD Score')
    plt.xlabel(p_col)
    plt.ylabel('IMD Score (Normalized)')

plt.tight_layout()
plt.savefig('outputs/figures/pollution_vs_deprivation.png', dpi=300)
plt.show()

In [ ]:
# Analyze environmental justice index
plt.figure(figsize=(10, 6))
sns.histplot(unified_df['env_justice_index'], bins=50, kde=True)
plt.title('Distribution of Environmental Justice Index')
plt.xlabel('Environmental Justice Index')
plt.ylabel('Frequency')
plt.savefig('outputs/figures/env_justice_distribution.png', dpi=300)
plt.show()

In [ ]:
# Identify areas with high environmental injustice
high_injustice = unified_df.sort_values('env_justice_index', ascending=False).head(20)
print("Top 20 areas with highest environmental injustice:")
high_injustice[['lsoa_code', 'lsoa_name', 'lad_name', 'imd_score_normalized', 
                'NO2_normalized', 'PM2.5_normalized', 'env_justice_index']]

## 4. Merging Health Data with Pollution Data

Let's merge the health indicators with pollution and deprivation data at the LAD level.

In [ ]:
# Aggregate unified data to LAD level
lad_aggregated = unified_df.groupby('lad_code').agg({
    'lad_name': 'first',
    'imd_score_normalized': 'mean',
    'NO2': 'mean',
    'O3': 'mean',
    'PM10': 'mean',
    'PM2.5': 'mean',
    'NO2_normalized': 'mean',
    'PM2.5_normalized': 'mean',
    'PM10_normalized': 'mean',
    'env_justice_index': 'mean'
}).reset_index()

# Merge with health data
merged_df = pd.merge(
    lad_aggregated,
    health_df,
    left_on='lad_code',
    right_on='local_authority_code',
    how='inner'
)

print(f"Merged dataset shape: {merged_df.shape}")
print(f"Number of LADs in merged dataset: {len(merged_df)}")

# Save merged dataset
merged_df.to_csv('outputs/data/lad_health_pollution_merged.csv', index=False)

# Display first few rows
merged_df.head()

## 5. Health-Pollution Relationship Analysis

Now, let's analyze the relationship between air pollution and health outcomes.

In [ ]:
# Calculate correlations
pollution_cols = ['NO2', 'O3', 'PM10', 'PM2.5']
health_cols = ['respiratory_health_index', 'chronic_conditions_normalized', 
              'asthma_diabetes_epilepsy_normalized', 'lrti_children_normalized']

print("Correlation between air pollution and health outcomes:")
for p_col in pollution_cols:
    for h_col in health_cols:
        corr, p_value = pearsonr(merged_df[p_col], merged_df[h_col])
        print(f"{p_col} vs {h_col}: r = {corr:.3f}, p = {p_value:.6f}")

In [ ]:
# Create scatter plots
plt.figure(figsize=(16, 12))

for i, p_col in enumerate(pollution_cols):
    plt.subplot(2, 2, i+1)
    sns.scatterplot(x=merged_df[p_col], y=merged_df['respiratory_health_index'], 
                    alpha=0.6, edgecolor=None)
    
    # Add trend line
    sns.regplot(x=merged_df[p_col], y=merged_df['respiratory_health_index'], 
               scatter=False, line_kws={"color": "red"})
    
    plt.title(f'{p_col} vs Respiratory Health Index')
    plt.xlabel(p_col)
    plt.ylabel('Respiratory Health Index')

plt.tight_layout()
plt.savefig('outputs/figures/pollution_vs_health.png', dpi=300)
plt.show()

In [ ]:
# Analyze double disadvantage (high pollution + high deprivation)
merged_df['double_disadvantage'] = (
    (merged_df['imd_score_normalized'] > merged_df['imd_score_normalized'].median()) & 
    ((merged_df['NO2_normalized'] > merged_df['NO2_normalized'].median()) | 
     (merged_df['PM2.5_normalized'] > merged_df['PM2.5_normalized'].median()))
)

# Compare health outcomes between double disadvantage and other areas
plt.figure(figsize=(12, 8))
sns.boxplot(x='double_disadvantage', y='respiratory_health_index', data=merged_df)
plt.title('Respiratory Health Index by Double Disadvantage Status')
plt.xlabel('Double Disadvantage (High Pollution + High Deprivation)')
plt.ylabel('Respiratory Health Index')
plt.xticks([0, 1], ['No', 'Yes'])
plt.savefig('outputs/figures/double_disadvantage_health.png', dpi=300)
plt.show()

In [ ]:
# T-test to compare means
from scipy.stats import ttest_ind
double_disadv = merged_df[merged_df['double_disadvantage']]['respiratory_health_index']
others = merged_df[~merged_df['double_disadvantage']]['respiratory_health_index']

t_stat, p_val = ttest_ind(double_disadv, others)
print(f"T-test comparing respiratory health index between double disadvantage areas and others:")
print(f"t-statistic: {t_stat:.3f}, p-value: {p_val:.6f}")
print(f"Mean for double disadvantage areas: {double_disadv.mean():.3f}")
print(f"Mean for other areas: {others.mean():.3f}")

## 6. Creating a Vulnerability Index

Let's create a composite vulnerability index combining pollution, deprivation, and health factors.

In [ ]:
# Select variables for the index
index_vars = [
    'imd_score_normalized', 'NO2_normalized', 'PM2.5_normalized', 
    'PM10_normalized', 'respiratory_health_index'
]

# Standardize variables
scaler = StandardScaler()
scaled_data = scaler.fit_transform(merged_df[index_vars])
scaled_df = pd.DataFrame(scaled_data, columns=index_vars)

# Create composite index (simple average)
merged_df['vulnerability_index'] = scaled_df.mean(axis=1)

# Normalize to 0-100 scale
min_val = merged_df['vulnerability_index'].min()
max_val = merged_df['vulnerability_index'].max()
merged_df['vulnerability_index'] = 100 * (merged_df['vulnerability_index'] - min_val) / (max_val - min_val)

In [ ]:
# Identify high vulnerability areas
high_vulnerability = merged_df.sort_values('vulnerability_index', ascending=False).head(20)
print("Top 20 areas with highest vulnerability:")
high_vulnerability[['lad_name', 'vulnerability_index', 'imd_score_normalized', 
                   'NO2_normalized', 'PM2.5_normalized', 'respiratory_health_index']]

In [ ]:
# Plot vulnerability index distribution
plt.figure(figsize=(10, 6))
sns.histplot(merged_df['vulnerability_index'], bins=30, kde=True)
plt.title('Distribution of Vulnerability Index')
plt.xlabel('Vulnerability Index (0-100)')
plt.ylabel('Frequency')
plt.savefig('outputs/figures/vulnerability_index_distribution.png', dpi=300)
plt.show()

# Save the full dataset with vulnerability index
merged_df.to_csv('outputs/data/lad_with_vulnerability_index.csv', index=False)

## 7. Cluster Analysis

Finally, let's perform cluster analysis to identify patterns in the data.

In [ ]:
# Select variables for clustering
cluster_vars = [
    'imd_score_normalized', 'NO2_normalized', 'PM2.5_normalized', 
    'respiratory_health_index', 'vulnerability_index'
]

# Standardize variables
scaler = StandardScaler()
scaled_data = scaler.fit_transform(merged_df[cluster_vars])

# Determine optimal number of clusters using the elbow method
inertia = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, 'o-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.savefig('outputs/figures/elbow_curve.png', dpi=300)
plt.show()

In [ ]:
# Choose k=4 clusters (this can be adjusted based on the elbow curve)
k = 4
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
merged_df['cluster'] = kmeans.fit_predict(scaled_data)

# Analyze clusters
cluster_summary = merged_df.groupby('cluster').agg({
    'lad_name': 'count',
    'imd_score_normalized': 'mean',
    'NO2_normalized': 'mean',
    'PM2.5_normalized': 'mean',
    'respiratory_health_index': 'mean',
    'vulnerability_index': 'mean'
}).rename(columns={'lad_name': 'count'})

print("Cluster Summary:")
cluster_summary

In [ ]:
# Visualize clusters using PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

plt.figure(figsize=(12, 8))
for i in range(k):
    plt.scatter(
        pca_result[merged_df['cluster'] == i, 0],
        pca_result[merged_df['cluster'] == i, 1],
        label=f'Cluster {i}'
    )

plt.title('PCA Visualization of Clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.savefig('outputs/figures/cluster_pca.png', dpi=300)
plt.show()

# Save clustered data
merged_df.to_csv('outputs/data/lad_clustered.csv', index=False)

## 8. Conclusion

In this analysis, we've explored the relationship between air pollution, socioeconomic deprivation, and respiratory health outcomes in the UK. We've identified areas of environmental injustice, created a vulnerability index, and performed cluster analysis to identify patterns in the data.

Key findings include:
1. The relationship between air pollution and deprivation
2. The impact of "double disadvantage" on respiratory health
3. Areas with high vulnerability that should be prioritized for interventions
4. Distinct clusters of areas with similar environmental justice and health profiles

These insights can inform policy decisions and targeted interventions to address environmental injustice and improve health outcomes in the most vulnerable communities.